### Importar Librerías y cargar datos

In [8]:
import numpy as np
import pandas as pd
import os

# Lista de posibles rutas
addresses = [
    'C:/Users/RONALD Q/OneDrive - LUZ DEL SUR S.A.A/Documentos/Estudios de Ingreso/ProyectoRyD_V2/Basededatos/DGA.xlsx',
    'C:/Users/roquispec/OneDrive - LUZ DEL SUR S.A.A/Documentos/Estudios de Ingreso/ProyectoRyD_V2/Basededatos/DGA.xlsx',
    'C:/Users/mticllacu/OneDrive - LUZ DEL SUR S.A.A/Archivos de Ronald Quispe Ocaña - ProyectoRyD_V2/Basededatos/DGA.xlsx'
]

df = None
for path in addresses:
    if os.path.exists(path):   # verifica si existe
        df = pd.read_excel(path,header=1)
        print(f"✅ Archivo cargado desde: {path}")
        break

if df is None:
    raise FileNotFoundError("❌ No se encontró el archivo en ninguna de las rutas especificadas.")

df.head()

✅ Archivo cargado desde: C:/Users/roquispec/OneDrive - LUZ DEL SUR S.A.A/Documentos/Estudios de Ingreso/ProyectoRyD_V2/Basededatos/DGA.xlsx


,Unnamed: 0,SERIE,FECHA DE MUESTRA,H2,CH4,C2H2,C2H4,C2H6,CO,CO2,O2,N2,TempR,HumR
0,NaN,D518293,2025-02-07 00:00:00,4.0,11.8,1.0,1.0,2.6,1064.6,3681.0,3372.2,28075.0,NaN,NaN
1,NaN,D518293,2023-08-29 00:00:00,3.0,9.0,1.0,1.0,2.0,788.0,3213.0,10157.0,60776.0,NaN,NaN
2,NaN,D518293,2022-06-14 00:00:00,5.0,9.0,1.0,1.0,2.0,758.0,3021.0,3106.0,20091.0,NaN,NaN
3,NaN,D518293,2022-04-28 00:00:00,1.0,8.0,1.0,1.0,2.0,745.0,2942.0,3837.0,34777.0,NaN,NaN
4,NaN,D518293,2021-08-15 00:00:00,7.0,7.0,1.0,1.0,1.0,547.0,2375.0,11922.0,50445.0,NaN,NaN


### Limpieza de datos

In [9]:
df["SERIE"] = df["SERIE"].astype(str)
df = df.drop(columns=['Unnamed: 0'])
# df["TENSION"] = df["TENSION"].str.split("/").str[0]
df = df.iloc[:, :10]
if 'FECHA DE MUESTRA' in df.columns:
    df = df.rename(columns={'FECHA DE MUESTRA': 'FECHA'})
elif 'FECHA DE\nMUESTRA' in df.columns:
    df = df.rename(columns={'FECHA DE\nMUESTRA': 'FECHA'})
df['FECHA'] = pd.to_datetime(df['FECHA'], errors='coerce')
df = df.dropna(subset=['FECHA'])
df_full = df.copy()
df.head()

,SERIE,FECHA,H2,CH4,C2H2,C2H4,C2H6,CO,CO2,O2
0,D518293,2025-02-07,4.0,11.8,1.0,1.0,2.6,1064.6,3681.0,3372.2
1,D518293,2023-08-29,3.0,9.0,1.0,1.0,2.0,788.0,3213.0,10157.0
2,D518293,2022-06-14,5.0,9.0,1.0,1.0,2.0,758.0,3021.0,3106.0
3,D518293,2022-04-28,1.0,8.0,1.0,1.0,2.0,745.0,2942.0,3837.0
4,D518293,2021-08-15,7.0,7.0,1.0,1.0,1.0,547.0,2375.0,11922.0


### Lógica

In [10]:
# ---------------------------
# LÍMITES Y GASES
# ---------------------------
limite = {
    'H2': {'concentracion': 150, 'tasa': 132},
    'CH4': {'concentracion': 130, 'tasa': 120},
    "C2H2": {"concentracion": 20, "tasa": 4},
    "C2H4": {"concentracion": 280, "tasa": 146},
    "C2H6": {"concentracion": 90, "tasa": 90},
    "CO": {"concentracion": 600, "tasa": 1060},
    "CO2": {"concentracion": 14000, "tasa": 10000}
}
gases2 = list(limite.keys())


### Columnas de lógica de concentración

In [11]:
# ---------------------------
# CONCENTRACIÓN
# ---------------------------
for gas in gases2:
    df[f'concentracion_{gas}'] = df.apply(lambda g: int(g[gas] > limite[gas]['concentracion']), axis=1)
df.head()

,SERIE,FECHA,H2,CH4,C2H2,C2H4,C2H6,CO,CO2,O2,concentracion_H2,concentracion_CH4,concentracion_C2H2,concentracion_C2H4,concentracion_C2H6,concentracion_CO,concentracion_CO2
0,D518293,2025-02-07,4.0,11.8,1.0,1.0,2.6,1064.6,3681.0,3372.2,0,0,0,0,0,1,0
1,D518293,2023-08-29,3.0,9.0,1.0,1.0,2.0,788.0,3213.0,10157.0,0,0,0,0,0,1,0
2,D518293,2022-06-14,5.0,9.0,1.0,1.0,2.0,758.0,3021.0,3106.0,0,0,0,0,0,1,0
3,D518293,2022-04-28,1.0,8.0,1.0,1.0,2.0,745.0,2942.0,3837.0,0,0,0,0,0,1,0
4,D518293,2021-08-15,7.0,7.0,1.0,1.0,1.0,547.0,2375.0,11922.0,0,0,0,0,0,0,0


### Columnas de lógica de tasa

In [12]:
# ---------------------------
# TASA DE INCREMENTO ANUAL
# ---------------------------
for gas in gases2:
    df[f"tasa_{gas}"] = df.groupby("SERIE").apply(
        lambda g: (g[gas].diff(-1) / ((g["FECHA"] - g["FECHA"].shift(-1)).dt.days) * 365)
    ).reset_index(level=0, drop=True)
df.head()

C:\Users\roquispec\AppData\Local\Temp\ipykernel_32840\3077718884.py:5: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df[f"tasa_{gas}"] = df.groupby("SERIE").apply(
C:\Users\roquispec\AppData\Local\Temp\ipykernel_32840\3077718884.py:5: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df[f"tasa_{gas}"] = df.groupby("SERIE").apply(
C:\Users\roquispec\AppData\Local\Temp\ipykernel_32840\3077718884.py:5: FutureWarning: DataFr

,SERIE,FECHA,H2,CH4,C2H2,C2H4,C2H6,CO,CO2,O2,...,concentracion_C2H6,concentracion_CO,concentracion_CO2,tasa_H2,tasa_CH4,tasa_C2H2,tasa_C2H4,tasa_C2H6,tasa_CO,tasa_CO2
0,D518293,2025-02-07,4.0,11.8,1.0,1.0,2.6,1064.6,3681.0,3372.2,...,0,1,0,0.691288,1.935606,0.0,0.0,0.414773,191.210227,323.522727
1,D518293,2023-08-29,3.0,9.0,1.0,1.0,2.0,788.0,3213.0,10157.0,...,0,1,0,-1.655329,0.000000,0.0,0.0,0.000000,24.829932,158.911565
2,D518293,2022-06-14,5.0,9.0,1.0,1.0,2.0,758.0,3021.0,3106.0,...,0,1,0,31.063830,7.765957,0.0,0.0,0.000000,100.957447,613.510638
3,D518293,2022-04-28,1.0,8.0,1.0,1.0,2.0,745.0,2942.0,3837.0,...,0,1,0,-8.554688,1.425781,0.0,0.0,1.425781,282.304688,808.417969
4,D518293,2021-08-15,7.0,7.0,1.0,1.0,1.0,547.0,2375.0,11922.0,...,0,0,0,-2.619617,1.746411,0.0,0.0,0.000000,107.404306,792.870813


### Asignación de puntajes

In [13]:
# ---------------------------
# FLAGS Y PUNTAJE
# ---------------------------
for gas, valores in limite.items():
    df[f"flag_{gas}"] = (df[f"tasa_{gas}"] > valores['tasa']).astype(int)

def puntaje_gas(row, gas):
    conc = row[f'concentracion_{gas}']
    tasa = row[f'flag_{gas}']
    if conc > 0 and tasa > 0:
        return 5
    elif conc > 0 and tasa <= 0:
        return 4
    elif conc <= 0 and tasa > 0:
        return 3
    else:
        return 1

for gas in gases2:
    df[f'puntaje_{gas}'] = df.apply(lambda row: puntaje_gas(row, gas), axis=1)
df.head()

,SERIE,FECHA,H2,CH4,C2H2,C2H4,C2H6,CO,CO2,O2,...,flag_C2H6,flag_CO,flag_CO2,puntaje_H2,puntaje_CH4,puntaje_C2H2,puntaje_C2H4,puntaje_C2H6,puntaje_CO,puntaje_CO2
0,D518293,2025-02-07,4.0,11.8,1.0,1.0,2.6,1064.6,3681.0,3372.2,...,0,0,0,1,1,1,1,1,4,1
1,D518293,2023-08-29,3.0,9.0,1.0,1.0,2.0,788.0,3213.0,10157.0,...,0,0,0,1,1,1,1,1,4,1
2,D518293,2022-06-14,5.0,9.0,1.0,1.0,2.0,758.0,3021.0,3106.0,...,0,0,0,1,1,1,1,1,4,1
3,D518293,2022-04-28,1.0,8.0,1.0,1.0,2.0,745.0,2942.0,3837.0,...,0,0,0,1,1,1,1,1,4,1
4,D518293,2021-08-15,7.0,7.0,1.0,1.0,1.0,547.0,2375.0,11922.0,...,0,0,0,1,1,1,1,1,1,1


### Cálculo de DGA

In [14]:
# ---------------------------
# CÁLCULO DGA
# ---------------------------
weight = {"C2H2":5,"H2":2,"CH4":3,"C2H4":4,"C2H6":3,"CO":2,"CO2":1}
total_weight = sum(weight.values())
df['DGA'] = df.apply(lambda row: sum(row[f'puntaje_{gas}']*weight[gas] for gas in gases2)/total_weight, axis=1)

df_DGA = df[['SERIE','FECHA','DGA']]
df_DGA

,SERIE,FECHA,DGA
0,D518293,2025-02-07,1.3
1,D518293,2023-08-29,1.3
2,D518293,2022-06-14,1.3
3,D518293,2022-04-28,1.3
4,D518293,2021-08-15,1.0
...,...,...,...
2674,338118,2013-02-11,1.3
2675,338118,2011-12-19,1.3
2676,338118,2011-11-28,1.0
2677,338118,2011-11-21,1.0


### Extensión de Calendario

### Transformación de datos y extensión del calendario desde 2015 al dia de hoy, esta fecha(2015) se puede cambiar, al hacerlo, se debe cambiar en todas las tablas(ACE,AIS,ARR,ARRdis,NUCiex,etc...)

In [15]:

# ---------------------------
# EXTENSIÓN DEL CALENDARIO DESDE 2025
# ---------------------------
inicio = "2015-01-01"
desde_2025 = f"{pd.Timestamp.today().year}-01-01"
fecha_inicio = pd.Timestamp(inicio)  # en el 2026 cambiar ---****
fecha_fin = pd.Timestamp.today().normalize()
fechas = pd.date_range(fecha_inicio, fecha_fin, freq="D")

# Calendario
todas_series = df['SERIE'].dropna().unique()
calendario = pd.MultiIndex.from_product([todas_series, fechas], names=["SERIE","FECHA"])
df_calendario = pd.DataFrame(index=calendario).reset_index()

# ---------- Tabla extendida de DGA ----------
ultimos_2024 = df_DGA[df_DGA['FECHA'] < fecha_inicio].sort_values('FECHA').groupby('SERIE').tail(1)
ultimos_2024['FECHA'] = fecha_inicio
base_ext = pd.concat([df_DGA, ultimos_2024], ignore_index=True)

df_extendida = pd.merge(df_calendario, base_ext, on=["SERIE","FECHA"], how="left")
df_extendida = df_extendida.groupby("SERIE").apply(lambda g: g.ffill()).reset_index(drop=True)

# ---------- Tabla extendida de detalles ----------
ultimos_2024_det = df_full[df_full['FECHA'] < fecha_inicio].sort_values('FECHA').groupby('SERIE').tail(1)
ultimos_2024_det['FECHA'] = fecha_inicio
base_ext_det = pd.concat([df_full, ultimos_2024_det], ignore_index=True)

df_extendida_detalles = pd.merge(df_calendario, base_ext_det, on=["SERIE","FECHA"], how="left")
df_extendida_detalles = df_extendida_detalles.groupby("SERIE").apply(lambda g: g.ffill()).reset_index(drop=True)
df_extendida_detalles.tail()

# ---------------------------
# DETALLES + DGA
# ---------------------------
df_detalles = pd.merge(df_full, df_DGA, on=["SERIE","FECHA"], how="left")
df_detalles_ext = pd.merge(df_extendida_detalles, df_extendida, on=["SERIE","FECHA"], how="left")

# Reordenar columnas: poner DGA después de FECHA
def reordenar(df_in):
    cols = list(df_in.columns)
    if "DGA" in cols:
        cols.remove("DGA")
        idx = cols.index("FECHA") + 1
        cols = cols[:idx] + ["DGA"] + cols[idx:]
    return df_in[cols]

df_detalles = reordenar(df_detalles)
df_detalles_ext = reordenar(df_detalles_ext)

C:\Users\roquispec\AppData\Local\Temp\ipykernel_32840\2794304994.py:21: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_extendida = df_extendida.groupby("SERIE").apply(lambda g: g.ffill()).reset_index(drop=True)
C:\Users\roquispec\AppData\Local\Temp\ipykernel_32840\2794304994.py:29: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_extendida_detalles = df_extendida_detalles.groupby("SERIE").apply(lambda g: g.ffill())

### Funciones a llamar 

In [16]:
# ---------------------------
# FUNCIONES PARA LLAMAR
# ---------------------------
def get_df_DGA():
    return df_DGA

def get_df_extendida_DGA():
    return df_extendida

def get_df_detalles_DGA():
    return df_detalles

def get_df_detalles_ext_DGA():
    return df_detalles_ext

print('\n ====== TABLA CON FECHAS ORIGINALES ====== \n')
print(get_df_DGA(), '\n')

print('\n ====== TABLA CON FECHAS EXTENDIDAS ====== \n')
print(get_df_extendida_DGA().head(), '\n')

print('\n ====== TABLA DE DETALLES DE DGA CON FECHAS ORIGINALES ====== \n')
print(get_df_detalles_DGA().head(), '\n')

print('\n ====== TABLA DE DETALLES DE DGA CON FECHAS EXTENDIDAS ====== \n')
print(get_df_detalles_ext_DGA().head(), '\n')


 ====== TABLA CON FECHAS ORIGINALES ====== 

        SERIE      FECHA  DGA
0     D518293 2025-02-07  1.3
1     D518293 2023-08-29  1.3
2     D518293 2022-06-14  1.3
3     D518293 2022-04-28  1.3
4     D518293 2021-08-15  1.0
...       ...        ...  ...
2674   338118 2013-02-11  1.3
2675   338118 2011-12-19  1.3
2676   338118 2011-11-28  1.0
2677   338118 2011-11-21  1.0
2678   338118 2011-09-17  1.0

[2676 rows x 3 columns] 


 ====== TABLA CON FECHAS EXTENDIDAS ====== 

    SERIE      FECHA  DGA
0  100138 2015-01-01  NaN
1  100138 2015-01-02  NaN
2  100138 2015-01-03  NaN
3  100138 2015-01-04  NaN
4  100138 2015-01-05  NaN 


 ====== TABLA DE DETALLES DE DGA CON FECHAS ORIGINALES ====== 

     SERIE      FECHA  DGA   H2   CH4  C2H2  C2H4  C2H6      CO     CO2  \
0  D518293 2025-02-07  1.3  4.0  11.8   1.0   1.0   2.6  1064.6  3681.0   
1  D518293 2023-08-29  1.3  3.0   9.0   1.0   1.0   2.0   788.0  3213.0   
2  D518293 2022-06-14  1.3  5.0   9.0   1.0   1.0   2.0   758.0  3021.0  

In [17]:
df_DGA[df_DGA['SERIE'] =='146916']

,SERIE,FECHA,DGA
564,146916,2025-02-06,1.3
565,146916,2023-08-31,1.3
566,146916,2022-08-16,1.4
567,146916,2022-08-04,1.3
568,146916,2022-04-11,1.3
569,146916,2022-01-07,1.3
570,146916,2021-07-06,1.3
571,146916,2020-06-16,1.3
572,146916,2019-04-23,1.3
573,146916,2018-08-01,1.3
